<a href="https://colab.research.google.com/github/kd303/trnsfrmr_pytrch_end_p1/blob/main/session_09_bert/bart_training/bart_training_paraphrasing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

  # BART for Paraphrasing with Simple Transformers

  Thanks to original author [Thilina Rajapakse](www.linkedin.com/in/t-rajapakse/) for producing the original work



In [3]:
data_dir = /content/drive/MyDrive/bart_data

!wget https://storage.googleapis.com/paws/english/paws_wiki_labeled_final.tar.gz -P /content/drive/MyDrive/bart_data

--2022-01-28 22:40:39--  https://storage.googleapis.com/paws/english/paws_wiki_labeled_final.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.128, 142.250.157.128, 142.251.8.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4687157 (4.5M) [application/gzip]
Saving to: ‘/content/drive/MyDrive/bart_data/paws_wiki_labeled_final.tar.gz’

paws_wiki_labeled_f 100%[===================>]   4.47M  10.7MB/s    in 0.4s    

2022-01-28 22:40:40 (10.7 MB/s) - ‘/content/drive/MyDrive/bart_data/paws_wiki_labeled_final.tar.gz’ saved [4687157/4687157]



In [2]:
data_dir = '/content/drive/MyDrive/bart_data'

In [8]:
!tar -xvf $data_dir/paws_wiki_labeled_final.tar.gz -C $data_dir

final/test.tsv
final/
final/train.tsv
final/dev.tsv


In [3]:
import warnings

import pandas as pd


def load_data(
    file_path, input_text_column, target_text_column, label_column, keep_label=1
):
    df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)
    df = df.loc[df[label_column] == keep_label]
    df = df.rename(
        columns={input_text_column: "input_text", target_text_column: "target_text"}
    )
    df = df[["input_text", "target_text"]]
    df["prefix"] = "paraphrase"

    return df


def clean_unnecessary_spaces(out_string):
    if not isinstance(out_string, str):
        warnings.warn(f">>> {out_string} <<< is not a string.")
        out_string = str(out_string)
    out_string = (
        out_string.replace(" .", ".")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ,", ",")
        .replace(" ' ", "'")
        .replace(" n't", "n't")
        .replace(" 'm", "'m")
        .replace(" 's", "'s")
        .replace(" 've", "'ve")
        .replace(" 're", "'re")
    )
    return out_string

In [4]:
import os
from datetime import datetime
import logging

import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

In [5]:
train_df = pd.read_csv("/content/drive/MyDrive/bart_data/final/train.tsv", sep="\t").astype(str)
eval_df = pd.read_csv("/content/drive/MyDrive/bart_data/final/dev.tsv", sep="\t").astype(str)

In [6]:
train_df = train_df.loc[train_df["label"] == "1"]
eval_df = eval_df.loc[eval_df["label"] == "1"]

train_df = train_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)
eval_df = eval_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)

train_df = train_df[["input_text", "target_text"]]
eval_df = eval_df[["input_text", "target_text"]]

train_df["prefix"] = "paraphrase"
eval_df["prefix"] = "paraphrase"

In [7]:
train_df = train_df[["prefix", "input_text", "target_text"]]
eval_df = eval_df[["prefix", "input_text", "target_text"]]

train_df = train_df.dropna()
eval_df = eval_df.dropna()

train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)

eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)

print(train_df)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


           prefix  ...                                        target_text
1      paraphrase  ...  The 1975 -- 76 season of the National Basketba...
3      paraphrase  ...  The results are high when comparable flow rate...
4      paraphrase  ...  It is the seat of the district of Zerendi in A...
5      paraphrase  ...  William Henry Harman was born in Waynesboro, V...
7      paraphrase  ...  Given a discrete set of probabilities formula ...
...           ...  ...                                                ...
49384  paraphrase  ...  The Romance language currently spoken in Galic...
49390  paraphrase  ...  It is necessary to note that k is a vector con...
49393  paraphrase  ...  Tim Tim Henman won 6 -- 2, 7 -- 6 against Yevg...
49395  paraphrase  ...  He was considered an active member of the Coun...
49397  paraphrase  ...  She was at Cork on 24 June, and arrived in the...

[21829 rows x 3 columns]


In [8]:
train_df.shape

(21829, 3)

In [9]:
train_df_1 = train_df[10214: ]

In [10]:
train_df_1.shape

(11615, 3)

In [1]:
help(Seq2SeqArgs)

NameError: ignored

In [14]:
!tar -xvf /content/drive/MyDrive/bart_data/paws_wiki_labeled_final.tar.gz

final/test.tsv
final/
final/train.tsv
final/dev.tsv


In [11]:
model_args = Seq2SeqArgs()
model_args.do_sample = True
model_args.eval_batch_size = 16 ## 64
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_length = 128
model_args.max_seq_length = 128
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.num_train_epochs = 2
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 16 ## 8
model_args.use_multiprocessing = False
model_args.wandb_project = "Paraphrasing with BART"


model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
)

model.train_model(train_df_1, eval_data=eval_df)

to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(eval_df["prefix"].tolist(), eval_df["input_text"].tolist())
]
truth = eval_df["target_text"].tolist()

preds = model.predict(to_predict)

# Saving the predictions if needed
os.makedirs("/content/drive/MyDrive/bart_data/prediction", exist_ok=True)



Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/971M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

  0%|          | 0/3539 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 0.5040417126975618}
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/best_model
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of facebook/bart-large model complete. Saved to outputs/.


Generating outputs:   0%|          | 0/222 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2343: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


FileNotFoundError: ignored

In [16]:
os.makedirs("/content/drive/MyDrive/bart_data/prediction", exist_ok=True)

In [18]:
len(preds)

3539

In [17]:
with open(f"/content/drive/MyDrive/bart_data/prediction/predictions_{datetime.now()}.txt", "w") as f:
    for i, text in enumerate(eval_df["input_text"].tolist()):
        f.write(str(text) + "\n\n")

        f.write("Truth:\n")
        f.write(truth[i] + "\n\n")

        f.write("Prediction:\n")
        for pred in preds[i]:
            f.write(str(pred) + "\n")
        f.write(
            "________________________________________________________________________________\n"
        )

In [28]:
import torch
torch.cuda.empty_cache()

In [29]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Fri Jan 28 23:42:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/

In [20]:
to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(eval_df["prefix"].tolist(), eval_df["input_text"].tolist())
]
truth = eval_df["target_text"].tolist()

preds = model.predict(to_predict)

# Saving the predictions if needed
os.makedirs("/content/drive/MyDrive/bart_data/prediction", exist_ok=True)

with open(f"/content/drive/MyDrive/bart_data/prediction/predictions_{datetime.now()}.txt", "w") as f:
    for i, text in enumerate(eval_df["input_text"].tolist()):
        f.write(str(text) + "\n\n")

        f.write("Truth:\n")
        f.write(truth[i] + "\n\n")

        f.write("Prediction:\n")
        for pred in preds[i]:
            f.write(str(pred) + "\n")
        # printing first 5 predidctions
        if i <= 4:
          print("Truth: " , truth[i])
          print("\n Prediction: \n", )
          for pred in preds[i]:
            print(str(pred) + "\n")
          print( "________________________________________________________________________________\n")
        f.write(
            "________________________________________________________________________________\n"
        )

Generating outputs:   0%|          | 0/222 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2343: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


Truth:  They were there for us to enjoy and they were there for us to pray.

 Prediction: 

Paraphrase : They were there to enjoy us and they were there for us to pray.

Paraphrase : They were there to enjoy us and they were there for us to pray.

Paraphrase : They were there to enjoy us, and they were there for us to pray.

________________________________________________________________________________

Truth:  In August, after the end of the war in June 1902, Higgins Southampton left the `` SSBavarian '' and returned to Cape Town the following month.

 Prediction: 

Paraphrase : After the end of the war in June 1902, Higgins left Southampton in the `` SSBavarian '' in August, returning to Cape Town the following month.

Paraphrase : After the end of the war in June 1902, Higgins left Southampton in the `` SSBavarian '' in August, returning to Cape Town the following month.

Paraphrase : After the end of the war in June 1902, Higgins left Southampton in the `` SSBavarian '' in August

In [ ]:
import logging

from simpletransformers.seq2seq import Seq2SeqModel


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

model = Seq2SeqModel(
    encoder_decoder_type="bart", encoder_decoder_name="outputs"
)


while True:
    original = input("Enter text to paraphrase: ")
    to_predict = [original]

    preds = model.predict(to_predict)

    print("---------------------------------------------------------")
    print(original)

    print()
    print("Predictions >>>")
    for pred in preds[0]:
        print(pred)

    print("---------------------------------------------------------")
    print()